In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2
import asyncio

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.core.series import Quote
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.core.utils import recognize_time

from qubx.impl.ccxt_connector import CCXTDataConnector
from qubx.impl.ccxt_trading import CCXTSyncTradingConnector
from qubx.utils.runner import get_account_config

def Q(time: str, bid: float, ask: float) -> Quote:
    return Quote(recognize_time(time), bid, ask, 0, 0)

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        
2024-07-12 18:42:17.820 [ ℹ️ ] NTP offset controller thread is started


2024-07-13 01:05:23.336 [ ⚠️ ] qubx.utils.ntp:__correct_offset:28 - time.windows.com NTP server request exception: [Errno 11001] getaddrinfo failed
2024-07-13 01:05:23.345 [ ⚠️ ] qubx.utils.ntp:__correct_offset:28 - pool.ntp.org NTP server request exception: [Errno 11001] getaddrinfo failed
2024-07-13 01:05:23.348 [ ⚠️ ] qubx.utils.ntp:__correct_offset:28 - europe.pool.ntp.org NTP server request exception: [Errno 11001] getaddrinfo failed
2024-07-13 01:05:23.352 [ ⚠️ ] qubx.utils.ntp:__correct_offset:28 - time.google.com NTP server request exception: [Errno 11001] getaddrinfo failed
2024-07-13 01:05:23.352 [ ❌ ] Unable to get ntp offset from neither of NTP servers list ['time.windows.com', 'pool.ntp.org', 'europe.pool.ntp.org', 'time.google.com']
2024-07-13 05:52:58.449 [ ⚠️ ] qubx.utils.ntp:__correct_offset:28 - time.windows.com NTP server request exception: [Errno 11001] getaddrinfo failed
2024-07-13 05:52:58.488 [ ⚠️ ] qubx.utils.ntp:__correct_offset:28 - pool.ntp.org NTP server req

In [2]:
class TestStrategy(IStrategy):
    capital_invested: int = 1000

    def on_start(self, ctx: StrategyContext):
        logger.info(f" -> Strategy started | {ctx.time()}")

    def on_fit(self, ctx: 'StrategyContext', fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f" -> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        logger.info(f" ||||| {event.time} -> {event} |||||")
        return None

    def ohlcs(self, timeframe:str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def on_stop(self, ctx: StrategyContext):
        logger.info(f" -> strategy is stopped")

    def tracker(self, ctx: StrategyContext) -> PortfolioRebalancerTracker:
        return PortfolioRebalancerTracker(ctx, 1000, 0)

# Simulated broker and paper trading

In [3]:
from qubx.backtester.simulator import SimulatedExchangeService

In [4]:
auth = get_account_config('binance-qbx', './configs/.env')

sbroker = SimulatedExchangeService('binance', 10000, 'vip0_usdt', 'USDT')
data = CCXTDataConnector('binance', sbroker, **auth)

instruments = [lookup.find_symbol('BINANCE', 'BTCUSDT')]
ctx = StrategyContext(
    TestStrategy(), dict(
        capital_invested=100,
        trading_allowed=False
    ), 
    data, sbroker, instruments=instruments,  # type: ignore
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=0),
    trigger_spec='1Min -5Sec',
)
S = ctx.strategy

2024-07-12 18:42:21.182 [ ℹ️ ] binance initialized - current time 1970-01-01T00:00:00.000000000
2024-07-12 18:42:21.183 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-12 18:42:21.184 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-12 18:42:21.185 [ 🐞 ] Triggering strategy on every 1Min bar after 55S
2024-07-12 18:42:21.183 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-12 18:42:21.184 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-12 18:42:21.185 [ 🐞 ] Triggering strategy on every 1Min bar after 55S


In [ ]:
ctx.start()

2024-07-12 18:42:22.955 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 0} for 
	['BTCUSDT'] 
2024-07-12 18:42:22.957 [ ℹ️ ] Subscribed on ohlc updates for 1 symbols: 
		['BTCUSDT']
2024-07-12 18:42:22.958 [ ℹ️ ]  -> Strategy started | 1970-01-01T00:00:00.000000000
2024-07-12 18:42:22.960 [ ℹ️ ] (StrategyContext) Start processing market data
2024-07-12 18:42:22.960 [ ℹ️ ] (StrategyContext) strategy is started


2024-07-12 18:42:31.634 [ 🐞 ] [2024-07-12T15:42:31.637422592]: Invoking TestStrategy on_fit('1970-01-01 00:00:00', 'None')
2024-07-12 18:42:31.634 [ ℹ️ ]  -> Fit is called | fit_time: 1970-01-01 00:00:00 / prev: None
2024-07-12 18:42:31.634 [ 🐞 ] [2024-07-12T15:42:31.637422592]: TestStrategy is fitted
2024-07-12 18:42:41.175 [ 🐞 ] [SIM-ORDER-BTCUSDT-100001 / TestStrategy_BTCUSDT_1] : MARKET BUY 0.1 of BTCUSDT  -> [CLOSED]
2024-07-12 18:42:41.175 [ 🐞 ] Executed 0.1 @ 58004.024999999994 of BTCUSDT for order SIM-ORDER-BTCUSDT-100001
2024-07-12 18:42:55.637 [ ℹ️ ]  ||||| 2024-07-12T15:42:55.603171328 -> TriggerEvent(time=numpy.datetime64('2024-07-12T15:42:55.603171328'), type='bar', instrument=Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE', base='BTC', quote='USDT', margin_symbol=None, min_tick=0.01, min_size_step=1e-05, min_size=1e-05, futures_info=None, _aux_instrument=None), data={o:57982.010000 | h:58024.000000 | l:57972.000000 | c:58005.990000 | v:683267.703702}

In [ ]:
data.get_historical_ohlcs('BTCUSDT', '5Min', 200)

In [11]:
print(sbroker.get_position(instruments[0]))
print(sbroker.get_orders())

2024-07-12 15:43:00 [BINANCE:BTCUSDT]       0.00000      0.00    +0.1965 | 58006.00        0.00
[]


In [8]:
ctx.trade('BTCUSDT', 0.1)

2024-07-12 18:42:41.169 [ ℹ️ ] (StrategyContext) sending market buy for 0.1 of BTCUSDT ...
2024-07-12 18:42:41.173 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100001 MARKET BUY 0.1 executed at 58004.024999999994
2024-07-12 18:42:41.174 [ ℹ️ ]   ::  traded 0.1 for BTCUSDT @ 58004.024999999994 -> 0.00
2024-07-12 18:42:41.175 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100001 MARKET BUY 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100001', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-12T15:42:40.359531520'), quantity=0.1, price=0, side='BUY', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_1', cost=0.0)

In [10]:
ctx.trade('BTCUSDT', -0.1)

2024-07-12 18:42:59.040 [ ℹ️ ] (StrategyContext) sending market sell for 0.1 of BTCUSDT ...
2024-07-12 18:42:59.041 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100002 MARKET SELL 0.1 executed at 58005.985
2024-07-12 18:42:59.041 [ ℹ️ ]   ::  traded -0.1 for BTCUSDT @ 58005.985 -> 0.20
2024-07-12 18:42:59.042 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100002 MARKET SELL 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100002', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-12T15:42:57.331495424'), quantity=0.1, price=0, side='SELL', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_2', cost=0.0)

In [124]:
ctx.trade('BTCUSDT', 0.1, 45000)

2024-07-11 08:55:43.969 [ ℹ️ ] (StrategyContext) sending limit buy for 0.1 of BTCUSDT ...
2024-07-11 08:55:43.969 [ 🐞 ] [OMS] BTCUSDT - registered SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 45000


Order(id='SIM-ORDER-BTCUSDT-100005', type='LIMIT', symbol='BTCUSDT', time=numpy.datetime64('2024-07-11T05:55:41.389935104'), quantity=0.1, price=45000, side='BUY', status='OPEN', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_5', cost=0.0)

In [136]:
ctx.cancel('BTCUSDT')

2024-07-11 08:56:31.011 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 canceled
2024-07-11 08:56:31.011 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 of BTCUSDT -> CANCELED
2024-07-11 08:56:31.011 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 of BTCUSDT -> CANCELED


In [81]:
ctx.stop()
data.close()

2024-07-11 12:12:14.148 [ ℹ️ ] > test is stopped
2024-07-11 12:12:14.149 [ ℹ️ ] 	CachedMarketDataHolder.update_by_bar took 0.0000254 secs
2024-07-11 12:12:14.149 [ ℹ️ ] 	StrategyContext._processing_bar took 0.0000359 secs
2024-07-11 12:12:14.150 [ ℹ️ ] 	StrategyContext.handler took 0.0000034 secs
2024-07-11 12:12:14.150 [ ℹ️ ] 	StrategyContext._process_incoming_data took 1.0039607 secs
2024-07-11 12:12:14.150 [ ℹ️ ] 	loggers.notify took 0.0000024 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	strategy.on_fit took 0.0014524 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	strategy.on_event took 0.0012006 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	StrategyContext.trade took 0.0033409 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	StrategyContext._processing_order took 0.0006063 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	StrategyContext._processing_deals took 0.0011736 secs
2024-07-11 12:12:14.151 [ ❌ ] This event loop is already running


c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\impl\ccxt_connector.py:288: RuntimeWarning: coroutine 'Exchange.close' was never awaited
  def get_quote(self, symbol: str) -> Optional[Quote]:


# Simulated data provider

In [22]:
from qubx.backtester.simulator import SimulatedExchangeService, SimulatedDataProvider, DataLoader

In [23]:
# r = CsvStorageDataReader("../tests/data/csv")
r = MultiQdbConnector('xlydian-data')

2024-07-12 18:49:58.281 [ 🐞 ] Connected to QuestDB at xlydian-data:8812


In [ ]:
# r.get_names('candles')

In [146]:
# instr = lookup.find_symbol("BINANCE.UM", "SOLUSDT")
# stream = r.read('SOLUSDT', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick, timestamp_units='ms'))
# stream = r.read('BINANCE.UM:SOLUSDT', '2024-04-01', '2024-04-10', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick, timestamp_units='ms'))
# stream = r.read('BINANCE.UM:BTCUSDT', '2024-01-01', '2024-02-10', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick, timestamp_units='ms'))
# stream[0], stream[-1]

In [24]:
sbroker = SimulatedExchangeService('binance', 10000, 'vip0_usdt', 'USDT', '2024-01-01 00:00:00')
sdp = SimulatedDataProvider('binance', sbroker, r)

instruments = [lookup.find_symbol('BINANCE.UM', s) for s in ['BTCUSDT', 'SOLUSDT', 'LINKUSDT']]
# sdp.subscribe('ohlc', instruments, '5Min')

ctx = StrategyContext(
    TestStrategy(), dict(
        capital_invested=100,
        trading_allowed=False
    ), 
    sdp, sbroker, instruments=instruments,  # type: ignore
    md_subscription=dict(type='ohlc', timeframe='5Min', nback=0),
    trigger_spec='5Min -1Sec',
    # trigger_spec="* * * * * *"

)
ctx.start()

2024-07-12 18:49:59.935 [ ℹ️ ] SimulatedData.binance initialized
2024-07-12 18:49:59.935 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-12 18:49:59.935 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-12 18:49:59.935 [ 🐞 ] Triggering strategy on every 5Min bar after 4Min59S
2024-07-12 18:49:59.943 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '5Min', 'nback': 0} for 
	['BTCUSDT', 'SOLUSDT', 'LINKUSDT'] 
2024-07-12 18:49:59.944 [ ℹ️ ]  -> Strategy started | 2024-01-01T00:00:00.000000000
2024-07-12 18:49:59.944 [ ℹ️ ] (StrategyContext) Start processing market data
2024-07-12 18:49:59.944 [ ℹ️ ] (StrategyContext) strategy is started
2024-07-12 18:49:59.935 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-12 18:49:59.935 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 1

In [25]:
# %%time
d = sdp.run('2024-01-01 00:00', '2024-01-01 00:30')

2024-07-12 18:50:01.966 [ 🐞 ] [2024-01-01T00:00:00.001000000]: Invoking TestStrategy on_fit('2024-01-01 00:00:00', 'None')
2024-07-12 18:50:01.968 [ ℹ️ ]  -> Fit is called | fit_time: 2024-01-01 00:00:00 / prev: None
2024-07-12 18:50:01.969 [ 🐞 ] [2024-01-01T00:02:00.000000000]: TestStrategy is fitted
2024-07-12 18:50:01.971 [ ℹ️ ]  ||||| 2024-01-01T00:04:59.999000000 -> TriggerEvent(time=numpy.datetime64('2024-01-01T00:04:59.999000000'), type='bar', instrument=Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None), data=None) |||||
2024-07-12 18:50:01.974 [ ℹ️ ]  ||||| 2024-01-01T00:09:59.999000000 -> TriggerEv

In [26]:
ctx.stop()

2024-07-12 18:50:04.594 [ ℹ️ ]  -> strategy is stopped
2024-07-12 18:50:04.595 [ ℹ️ ] 	CachedMarketDataHolder.update_by_bar took 0.0000691 secs
2024-07-12 18:50:04.595 [ ℹ️ ] 	StrategyContext._processing_bar took 0.0001002 secs
2024-07-12 18:50:04.596 [ ℹ️ ] 	StrategyContext.handler took 0.0000029 secs
2024-07-12 18:50:04.597 [ ℹ️ ] 	strategy.on_fit took 0.0023251 secs
2024-07-12 18:50:04.597 [ ℹ️ ] 	StrategyContext._process_incoming_data took 2.2218490 secs
2024-07-12 18:50:04.598 [ ℹ️ ] 	loggers.notify took 0.0000023 secs
2024-07-12 18:50:04.599 [ ℹ️ ] 	StrategyContext.trade took 0.0028840 secs
2024-07-12 18:50:04.599 [ ℹ️ ] 	StrategyContext._processing_order took 0.0014158 secs
2024-07-12 18:50:04.600 [ ℹ️ ] 	StrategyContext._processing_deals took 0.0005753 secs
2024-07-12 18:50:04.600 [ ℹ️ ] 	strategy.on_event took 0.0008872 secs
2024-07-12 18:50:04.601 [ ℹ️ ] 	CachedMarketDataHolder.update_by_quote took 0.0000142 secs
2024-07-12 18:50:04.595 [ ℹ️ ] 	CachedMarketDataHolder.update_b

In [103]:
str(sbroker.get_position(instruments[0]))

'2024-01-01 00:19:30 [BINANCE.UM:BTCUSDT]       0.000       0.0    +0.0000 | 42549.1        0.00'

In [13]:
from queue import Queue, Empty
from time import monotonic as time
from threading import Thread
from multiprocessing import Process, JoinableQueue

class PeekQueue(Queue):
    def peek(self, block=True, timeout=None):
        '''Return first item from queue without removing it

        If optional args 'block' is true and 'timeout' is None (the default),
        block if necessary until an item is available. If 'timeout' is
        a non-negative number, it blocks at most 'timeout' seconds and raises
        the Empty exception if no item was available within that time.
        Otherwise ('block' is false), return an item if one is immediately
        available, else raise the Empty exception ('timeout' is ignored
        in that case).
        '''
        with self.not_empty:
            if not block:
                if not self._qsize():
                    raise Empty
            elif timeout is None:
                while not self._qsize():
                    self.not_empty.wait()
            elif timeout < 0:
                raise ValueError("'timeout' must be a non-negative number")
            else:
                endtime = time() + timeout
                while not self._qsize():
                    remaining = endtime - time()
                    if remaining <= 0.0:
                        raise Empty
                    self.not_empty.wait(remaining)
            item = self.queue[0]     #<----------- THE ONLY CHANGE from get()
            self.not_full.notify()
            return item


In [165]:
# q = PeekQueue(1)
q = JoinableQueue()

def picker():
    while True:
        v = q.get()
        print(f"Peek {v}", flush=True)
        if v is None:
            print(f"Break ...", flush=True)
            break
        q.task_done()
    q.task_done()
    print(f"Finish ...", flush=True)

def getter():
    while True:
        v = q.get()
        print(f"Got {v}", flush=True)
        if v is None:
            break
        q.task_done()
    q.task_done()


t1 = Thread(target=picker, daemon=True)
t2 = Thread(target=getter, daemon=True)

In [166]:
t2.start()
t1.start()

In [167]:
q.put(0)

Got 0
Peek 1
Got 1
Peek 1
Got 1
Peek 1
Got 1
Peek 1
Got 1
Peek None
Break ...
Finish ...
Got None


In [ ]:
q.put(1)

In [178]:
q.put(None)

In [179]:
q.join()

In [ ]:
from threading import Thread
from multiprocessing import Process, JoinableQueue
import time
import os

q = JoinableQueue()

def producer():
    for item in range(30):
        time.sleep(0.5)
        q.put(item)
    pid = os.getpid()
    print(f'producer {pid} done', flush=True)


def worker():
    while True:
        item = q.get()
        pid = os.getpid()
        print(f'pid {pid} Working on {item}', flush=True)
        print(f'pid {pid} Finished {item}', flush=True)
        q.task_done()

for i in range(5):
    p = Thread(target=worker, daemon=True).start()

producers = []
# it is easy to extend it to multi producers.
for i in range(1):
    p = Thread(target=producer)
    producers.append(p)
    p.start()

# make sure producers done
for p in producers:
    p.join()

# block until all workers are done
# q.join()
# print('All work completed')